In [41]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import nltk 
import bs4 
import re 
import string 
from collections import Counter 
from nltk.tokenize import word_tokenize 

In [42]:
data_link = 'D:/Pyhton/Autocorrect/auto_correct.txt'

we have an artical, now we need to get all words in this artical in one list 

In [43]:
def load_words(data_file):
    # create list to get all words 
    words =[]
    with open(data_file ,'r') as file:
        load_file = file.read()
        # ignore all white space 
        process_file = re.findall(r'\w+' , load_file)
        for word in process_file :
            words.append(word.lower())
    return words

loading data

In [44]:
data = load_words(data_link)
data[:10]

['this',
 'is',
 'the',
 '100th',
 'etext',
 'file',
 'presented',
 'by',
 'project',
 'gutenberg']

create our vocabular --> remove repeated words

In [45]:
vocab = set(data)
print(f'number of words before: {len(data)}')
print(f'number of words after: {len(vocab)}')
print(f'number of repeated: {len(data) - len(vocab)}')

number of words before: 929396
number of words after: 23902
number of repeated: 905494


Word Count
we need to get counts of words to calculate probabilities of each word
let's get an example for what we get from get_count function
Key	Value
I	2
am	2
happy	1
running	1

In [46]:
def get_count(word_list):
    final_dict = dict()
    final_dict = Counter(word_list)
    return final_dict 

# let's get it now 


count_words = get_count(data)

In [47]:
print(f'| key\tvalue\t|')
print(f'| ---\t-----\t|')
for i in range(6):
    get_keys =list( count_words.keys())
    print(f'| {get_keys[i]}\t{count_words[f"{get_keys[i]}"]}\t|')

| key	value	|
| ---	-----	|
| this	6853	|
| is	9784	|
| the	27660	|
| 100th	1	|
| etext	245	|
| file	20	|


Get Probability

now it's time for get probability to each word 
now we have dictionary of {words: count }
and we have length of all words in artical to get probability of each of them.

In [48]:
def get_proba(count_words):
    proba_dict = dict()
    total_words = sum(count_words.values())
    for word , word_value in count_words.items():
        proba_dict[word] = word_value / total_words 
    return proba_dict 


words_proba = get_proba(count_words)

In [49]:
print(f'| word\t|probability\t\t|')
print(f'| ------|-----------------------|')
for i in range(6):
    get_keys =list( words_proba.keys())
    print(f'| {get_keys[i]}\t|{words_proba[f"{get_keys[i]}"]}\t|')

| word	|probability		|
| ------|-----------------------|
| this	|0.007373606083951298	|
| is	|0.01052726717136721	|
| the	|0.029761264304989477	|
| 100th	|1.0759676176785783e-06	|
| etext	|0.00026361206633125167	|
| file	|2.1519352353571567e-05	|


Performing String Manipulations
Now, that we have computed P(wi)
 for all the words in the corpus, we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. In this section, we will implement four functions:

delete_letter: given a word, it returns all the possible strings that have one character removed.
switch_letter: given a word, it returns all the possible strings that have two adjacent letters switched.
replace_letter: given a word, it returns all the possible strings that have one character replaced by another different letter.
insert_letter: given a word, it returns all the possible strings that have an additional character inserted.

let's start with delete_letter

In [50]:
def delete_letter(word , verbos = False):
    # to explain what will happen 
    # we just need two lists 
    # 1. split list ---> return a tuple of ( 'c' , 'ans') 
    # 2. delete list ---> return a word deleted one charater each time 
    
    
    split_list  = [(word[:i], word[i:]) for i in range(len(word))]
    delete_list = [L+C[1:] for L,C in split_list]
    
    if verbos:
        print(f'word:{word}\nsplit list: {split_list}\ndelete list: {delete_list}')
    return delete_list 

checking our function

In [51]:
get_items = delete_letter(word='cans' , verbos=True)

word:cans
split list: [('', 'cans'), ('c', 'ans'), ('ca', 'ns'), ('can', 's')]
delete list: ['ans', 'cns', 'cas', 'can']


second function is switching

In [52]:
def switching_letter(word , verbos= False):
    
    split_list = [(word[:i],word[i:]) for i in range(len(word))]

    switch_list= [L+R[1]+R[0]+R[2:] for L,R in split_list  if len(R)>=2]
    
    if verbos :
        print(f'word: {word}\nsplit list:{split_list}\nswitching list: {switch_list}')
    return switch_list 

checking our function 

In [53]:
get_items = switching_letter(word='ate' , verbos = True)

word: ate
split list:[('', 'ate'), ('a', 'te'), ('at', 'e')]
switching list: ['tae', 'aet']


replace operation

In [54]:
def replace(word  , verbos = False):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list  =[ (word[0:i] , word[i:]) for i in range(len(word))  ]
    replace_list=[ L+ letter+(R[1:] if len(R)>1 else '') for L,R in split_list for letter in letters    ]
    replace_list= sorted( set(replace_list))
    if verbos:
        print(f'word: {word}\nsplit list: {split_list}\nreplace list: {replace_list}')
    return replace_list

checking our function 

In [55]:
get_items = replace(word='can' , verbos=True)

word: can
split list: [('', 'can'), ('c', 'an'), ('ca', 'n')]
replace list: ['aan', 'ban', 'caa', 'cab', 'cac', 'cad', 'cae', 'caf', 'cag', 'cah', 'cai', 'caj', 'cak', 'cal', 'cam', 'can', 'cao', 'cap', 'caq', 'car', 'cas', 'cat', 'cau', 'cav', 'caw', 'cax', 'cay', 'caz', 'cbn', 'ccn', 'cdn', 'cen', 'cfn', 'cgn', 'chn', 'cin', 'cjn', 'ckn', 'cln', 'cmn', 'cnn', 'con', 'cpn', 'cqn', 'crn', 'csn', 'ctn', 'cun', 'cvn', 'cwn', 'cxn', 'cyn', 'czn', 'dan', 'ean', 'fan', 'gan', 'han', 'ian', 'jan', 'kan', 'lan', 'man', 'nan', 'oan', 'pan', 'qan', 'ran', 'san', 'tan', 'uan', 'van', 'wan', 'xan', 'yan', 'zan']


In [56]:
def insert(word , verbos =False):
    
    letters = 'abcdefghijklmnopqrstuvwxyz'
    split_list = [(word[:i] , word[i:]) for i in range(len(word)+1)]
    insert_list= [L+ letter+R for L,R in split_list for letter in letters]
    insert_list = sorted( set(insert_list)  )
    if verbos:
        print(f'word:{word}\nsplit list:{split_list}\ninsert list:{insert_list}')
    return insert_list


checking our function 

In [57]:
get_items  = insert(word='at' , verbos=True)

word:at
split list:[('', 'at'), ('a', 't'), ('at', '')]
insert list:['aat', 'abt', 'act', 'adt', 'aet', 'aft', 'agt', 'aht', 'ait', 'ajt', 'akt', 'alt', 'amt', 'ant', 'aot', 'apt', 'aqt', 'art', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'aut', 'avt', 'awt', 'axt', 'ayt', 'azt', 'bat', 'cat', 'dat', 'eat', 'fat', 'gat', 'hat', 'iat', 'jat', 'kat', 'lat', 'mat', 'nat', 'oat', 'pat', 'qat', 'rat', 'sat', 'tat', 'uat', 'vat', 'wat', 'xat', 'yat', 'zat']


Edit One Letter

In [58]:
def edit_one_letter(word , switch = True):
    
    edit_one_set = set()
    edit_one_set.update(delete_letter(word))
    if switch:
        edit_one_set.update(switching_letter(word))
    edit_one_set.update(replace(word))
    edit_one_set.update(insert(word))
    if word in edit_one_set:
        edit_one_set.remove(word)
    return edit_one_set

checking our function 

In [59]:
word = 'at'
word_edit = edit_one_letter(word=word)
listed = sorted( list(word_edit))


print(f'word:{word}:\nedit one letter: {listed}' )

word:at:
edit one letter: ['a', 'aa', 'aat', 'ab', 'abt', 'ac', 'act', 'ad', 'adt', 'ae', 'aet', 'af', 'aft', 'ag', 'agt', 'ah', 'aht', 'ai', 'ait', 'aj', 'ajt', 'ak', 'akt', 'al', 'alt', 'am', 'amt', 'an', 'ant', 'ao', 'aot', 'ap', 'apt', 'aq', 'aqt', 'ar', 'art', 'as', 'ast', 'ata', 'atb', 'atc', 'atd', 'ate', 'atf', 'atg', 'ath', 'ati', 'atj', 'atk', 'atl', 'atm', 'atn', 'ato', 'atp', 'atq', 'atr', 'ats', 'att', 'atu', 'atv', 'atw', 'atx', 'aty', 'atz', 'au', 'aut', 'av', 'avt', 'aw', 'awt', 'ax', 'axt', 'ay', 'ayt', 'az', 'azt', 'bat', 'bt', 'cat', 'ct', 'dat', 'dt', 'eat', 'et', 'fat', 'ft', 'gat', 'gt', 'hat', 'ht', 'iat', 'it', 'jat', 'jt', 'kat', 'kt', 'lat', 'lt', 'mat', 'mt', 'nat', 'nt', 'oat', 'ot', 'pat', 'pt', 'qat', 'qt', 'rat', 'rt', 'sat', 'st', 't', 'ta', 'tat', 'tt', 'uat', 'ut', 'vat', 'vt', 'wat', 'wt', 'xat', 'xt', 'yat', 'yt', 'zat', 'zt']


In [60]:
def edit_two_letter(word , switch=True):
    
    edit_two_set =set()
    edit_one = edit_one_letter(word , switch)
    for W in edit_one:
        edit_two = edit_one_letter(W,switch)
        edit_two_set.update(edit_two)
    return edit_two_set 

In [61]:
word = 'a'
editing = edit_two_letter(word = word)
final_words = sorted( list(editing))
print(f'word:{word}\nEdit Two Letters:\n{final_words}')

word:a
Edit Two Letters:
['', 'a', 'aa', 'aaa', 'aab', 'aac', 'aad', 'aae', 'aaf', 'aag', 'aah', 'aai', 'aaj', 'aak', 'aal', 'aam', 'aan', 'aao', 'aap', 'aaq', 'aar', 'aas', 'aat', 'aau', 'aav', 'aaw', 'aax', 'aay', 'aaz', 'ab', 'aba', 'abb', 'abc', 'abd', 'abe', 'abf', 'abg', 'abh', 'abi', 'abj', 'abk', 'abl', 'abm', 'abn', 'abo', 'abp', 'abq', 'abr', 'abs', 'abt', 'abu', 'abv', 'abw', 'abx', 'aby', 'abz', 'ac', 'aca', 'acb', 'acc', 'acd', 'ace', 'acf', 'acg', 'ach', 'aci', 'acj', 'ack', 'acl', 'acm', 'acn', 'aco', 'acp', 'acq', 'acr', 'acs', 'act', 'acu', 'acv', 'acw', 'acx', 'acy', 'acz', 'ad', 'ada', 'adb', 'adc', 'add', 'ade', 'adf', 'adg', 'adh', 'adi', 'adj', 'adk', 'adl', 'adm', 'adn', 'ado', 'adp', 'adq', 'adr', 'ads', 'adt', 'adu', 'adv', 'adw', 'adx', 'ady', 'adz', 'ae', 'aea', 'aeb', 'aec', 'aed', 'aee', 'aef', 'aeg', 'aeh', 'aei', 'aej', 'aek', 'ael', 'aem', 'aen', 'aeo', 'aep', 'aeq', 'aer', 'aes', 'aet', 'aeu', 'aev', 'aew', 'aex', 'aey', 'aez', 'af', 'afa', 'afb', 'afc'

Make Suggestions

In [62]:
def make_suggestions(word, proba ,vocab ,n=2 ,verbos=False):
    suggestions = []
    top_n_suggestions=[]
    
    suggestions = list( (word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letter(word).intersection(vocab) )
    top_sug = [[s,proba[s]] for s in list(suggestions)  ]
    top_proba = max([proba[s] for s in suggestions])
    top_n_suggestions = top_sug[:n]
    if verbos:
        print(f'word: {word}\nSuggestions:\n{suggestions}')
        print(f'top probability :{top_proba}')
    return top_n_suggestions

showing result

In [63]:
word  = 'dys'
proba = words_proba
vocab = vocab
n     = 2
verbos= True

get_items = make_suggestions(word, proba, vocab ,n,verbos)
print(f'top 2 : \n{get_items}')

word: dys
Suggestions:
['des', 'days', 'dy', 'dis', 'dye']
top probability :0.00022487723209482286
top 2 : 
[['des', 3.227902853035735e-06], ['days', 0.00022487723209482286]]
